In [1]:
import pickle
from util.params import Params
from util.env import set_param
from main import Main
import torch
from models.GDN import GDN
from test_loop import test
from train_loop import train
import pandas as pd
from util.data import getAttacks
from evaluate import createMetrics
import pickle
import matplotlib.pyplot as plt
import os
def plot_with_full_row(df:pd.DataFrame,attacks,run):
    num_plots = len(attacks)  # Number of data points
    num_rows = (num_plots + 1) // 2  # Two columns per row

    # Create a figure with a grid layout, including space for the full-row plot
    fig = plt.figure(figsize=(12, 6 + 5 * num_rows))  # Adjust overall size
    grid = fig.add_gridspec(num_rows + 1, 2)  # Add one row for the overarching plot

    # Full-row plot at the top
    ax_full_row = fig.add_subplot(grid[0, :])  # Span across all columns
    df.plot(ax=ax_full_row)
    ax_full_row.set_title(f"all attacks at run: {run}")
    ax_full_row.legend()

    # Individual subplots below
    for i, attack in enumerate(attacks):
        left=(attack[0]//100)*100
        right=((attack[1]//100)+1)*100
        row, col = divmod(i, 2)  # Determine row and column for the subplot
        ax = fig.add_subplot(grid[row + 1, col])  # Offset by one for the full-row plot
        df.loc[left:right].plot(ax=ax)  # Example plot
        ax.set_title(f"attack {i+1}")

    # Adjust layout to prevent overlap
    plt.tight_layout()
    plt.show()

In [2]:
main_path="./snapshot/gdn_swat/25_04_04_21_13_20"
count=1
thr=1

In [3]:
for run in range(count):
  path = f"{main_path}/{run}/"
  if not os.path.exists(f"{path}param.pickle"):
    print(f"Skipping run {run}")
    continue
  param: Params = pickle.load(file=open(f"{path}param.pickle", "rb"))
  mode_dict=pickle.load(file=open(f"{path}model_parameters.pickle", "rb"))
  param.val_ratio=0
  set_param(param)
  # print(param.summary())
  main = Main(param=param,modelParams=mode_dict)

  # print(len(main.train_dataloader),len(main.val_dataloader),len(main.test_dataloader))
  train_avg_loss, train_result = test(main.model, main.train_dataloader)
  # print("train_avg_loss: ",train_avg_loss)
  # val_avg_loss, val_result = test(main.model, main.val_dataloader)
  # val_avg_loss
  test_avg_loss, test_result = test(
              main.model, main.test_dataloader
          )
  # print("test_avg_loss: ",test_avg_loss)
  # print("max: ",test_result[0].max(),
  #     #   val_result[0].max(),
  #       train_result[0].max())
  # print("min: ",test_result[0].min(),
  #     #   val_result[0].min(),
  #       train_result[0].min())

  minimum=0#val_result[0].min()
  maximum=train_result[0].max().item()*thr#*24.5

  pred_normalized=torch.where((test_result[0]>maximum) ,torch.tensor(1),torch.tensor(0)).cpu().numpy()

  labels_df=pd.DataFrame({"pred":test_result[0].cpu().numpy().squeeze(-1),
                          "pred_filterd":pred_normalized.squeeze(-1),
                          "truth":test_result[2].cpu().squeeze(-1).numpy()})
  attacks=getAttacks(labels_df,"truth")
  plot_with_full_row(labels_df,attacks,run)

# DATASET 

*Datasets.swat*
#####################################
sensors count:  31
actuators count:  9
consts count:  12
consts:  {'P102': 1.0, 'P201': 1.0, 'P202': 1.0, 'P204': 1.0, 'P206': 1.0, 'P401': 1.0, 'P403': 1.0, 'P404': 1.0, 'P502': 1.0, 'P601': 1.0, 'P603': 1.0, 'attack': 0.0}
#####################################
['FIT101', 'LIT101', 'MV101', 'AIT201', 'AIT202', 'AIT203', 'FIT201', 'MV201', 'DPIT301', 'FIT301', 'LIT301', 'MV301', 'MV302', 'MV303', 'MV304', 'AIT401', 'AIT402', 'FIT401', 'LIT401', 'AIT501', 'AIT502', 'AIT503', 'AIT504', 'FIT501', 'FIT502', 'FIT503', 'FIT504', 'PIT501', 'PIT502', 'PIT503', 'FIT601']
Model is trained. Loading from file .....


ValueError: too many values to unpack (expected 2)

In [ ]:
k=createMetrics(test_result,maximum)
print(k)
k.keys()

In [ ]:
k2=['TP', 'FP', 'TN', 'FN']
k1=["Accuracy","Precision","Recall","F1"]
d1={key:[] for key in k1}
d2={key:[] for key in k2}
x=[]
for i in range(100):
    _max=maximum*(.5+i/100)
    rr=createMetrics(test_result,_max)
    x.append(_max)
    for key  in d1.keys():
        d1[key].append(float(rr[key]))
    for key  in d2.keys():
        d2[key].append(float(rr[key]))

In [ ]:
df1=pd.DataFrame(d1,index=x)
df1.plot()

In [ ]:
df2=pd.DataFrame(d2,index=x)
df2.plot()